# Predicting Future Trends in Sea Surface Temperature

In [ ]:
# !pip install --upgrade google-cloud-bigquery

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
from google.colab import auth
from google.cloud import bigquery


In [ ]:
# auth.authenticate_user()

# project_id = 'bamboo-medium-450316-m8'
# client = bigquery.Client(project=project_id)


In [ ]:
!pip install copernicusmarine

# Step 1: Install the Copernicus Marine Toolbox
import copernicusmarine

# Step 2: Login with Your Copernicus Marine Credentials
copernicusmarine.login(username="email", password="password")


In [ ]:
# Subset

copernicusmarine.subset(
   dataset_id = "METOFFICE-GLO-SST-L4-NRT-OBS-SST-V2",
   variables = ["analysed_sst"],
   start_datetime = "2011-01-01T00:00:00",
   end_datetime = "2024-12-31T23:59:59",
   minimum_longitude = -59.75,
   maximum_longitude = -59.40,
   minimum_latitude = 12.95,
   maximum_latitude = 13.25,
   minimum_depth = 0,
   maximum_depth = 1,
   output_filename = "sst_data_for_major_locations.nc",
   output_directory = "copernicus-data"
)

In [ ]:
import xarray as xr

# Open the NetCDF file
ds = xr.open_dataset('/content/copernicus-data/sst_data_for_major_locations.nc')
print(ds)

In [ ]:
# Flatten and convert to dataframe
df = ds['analysed_sst'].to_dataframe().reset_index()

In [ ]:
df.head()

# Data Exploration

In [ ]:
df.shape

In [ ]:
import numpy as np

# Load coordinate arrays from dataset
latitudes = ds.latitude.values
longitudes = ds.longitude.values

# Known coastal coordinates (approximate)
locations = {
    "Crane Beach": (13.1089, -59.4414),
    "Oistins": (13.0647, -59.5515),
    "Conset Bay": (13.2025, -59.4827)
}

# Find nearest available offshore grid point for each location
offshore_matches = {}

for name, (lat, lon) in locations.items():
    closest_lat = latitudes[np.abs(latitudes - lat).argmin()]
    closest_lon = longitudes[np.abs(longitudes - lon).argmin()]
    offshore_matches[name] = (closest_lat, closest_lon)

offshore_matches


In [ ]:
# Extract SST for each offshore location
location_sst = {}

for name, (lat_val, lon_val) in offshore_matches.items():
    sst = ds['analysed_sst'].sel(latitude=lat_val, longitude=lon_val).to_dataframe().reset_index()
    sst['location'] = name  # Add location column
    location_sst[name] = sst

# Combine all locations into one DataFrame
import pandas as pd
df_combined = pd.concat(location_sst.values(), ignore_index=True)

# Preview
df_combined.head()


## Handle missing values

In [ ]:
df_combined.head()

In [ ]:
df_combined.isnull().sum()

### 1. Line Plot: Sea Surface Temperature Over Time

In [ ]:
import matplotlib.pyplot as plt

df_combined['time'] = pd.to_datetime(df_combined['time'])

# Group by date (ignore time part) and calculate mean SST
df_daily_avg = df_combined.groupby(df_combined['time'].dt.date)['analysed_sst'].mean().reset_index()

plt.figure(figsize=(10, 5))
plt.plot(df_daily_avg['time'], df_daily_avg['analysed_sst'])
plt.title('Average Daily Sea Surface Temperature')
plt.xlabel('Date')
plt.ylabel('Avg Analysed SST')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### 2. Histogram: Distribution of Sea Surface Temperature

In [ ]:
plt.figure(figsize=(8, 4))
plt.hist(df_combined['analysed_sst'], bins=30, edgecolor='black')
plt.title('Distribution of Analysed SST')
plt.xlabel('Analysed SST')
plt.ylabel('Frequency')
plt.show()


### 3. Boxplot: SST Distribution by Location

In [ ]:
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.boxplot(data=df_combined, x='location', y='analysed_sst')
plt.title('SST Distribution by Location')
plt.xticks(rotation=45)
plt.show()



# Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np

# Convert date to ordinal for regression
df_daily_avg['date_ordinal'] = pd.to_datetime(df_daily_avg['time']).map(pd.Timestamp.toordinal)

# Fit linear regression
model = LinearRegression()
model.fit(df_daily_avg[['date_ordinal']], df_daily_avg['analysed_sst'])

# Predict values
df_daily_avg['trend_line'] = model.predict(df_daily_avg[['date_ordinal']])

# Plot
plt.figure(figsize=(10, 5))
plt.plot(df_daily_avg['time'], df_daily_avg['analysed_sst'], label='Daily Avg SST', alpha=0.6)
plt.plot(df_daily_avg['time'], df_daily_avg['trend_line'], label='Linear Trend Line', color='green', linewidth=2)
plt.title('Average Daily SST with Linear Trend')
plt.xlabel('Date')
plt.ylabel('Analysed SST')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
from datetime import timedelta

# Extend the date range by 5 years from the last date
last_date = df_daily_avg['time'].max()
future_dates = pd.date_range(start=last_date + timedelta(days=1), periods=5*365, freq='D')  # 5 years

# Convert to ordinal
future_ordinals = future_dates.map(pd.Timestamp.toordinal).values.reshape(-1, 1)

# Predict SST for future dates
future_sst = model.predict(future_ordinals)

# Combine into a new DataFrame
future_df = pd.DataFrame({'time': future_dates, 'predicted_sst': future_sst})

# Plot historical + forecast
plt.figure(figsize=(12, 6))
plt.plot(df_daily_avg['time'], df_daily_avg['analysed_sst'], label='Historical SST', alpha=0.5)
plt.plot(df_daily_avg['time'], df_daily_avg['trend_line'], label='Historical Trend', color='green')
plt.plot(future_df['time'], future_df['predicted_sst'], label='Forecast SST (Next 5 Years)', color='red')
plt.title('SST Forecast for Next 5 Years')
plt.xlabel('Date')
plt.ylabel('Analysed SST')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


# Non Linear Model

In [ ]:
!pip install prophet

In [ ]:
from prophet import Prophet

# Prepare data
prophet_df = df_daily_avg[['time', 'analysed_sst']].rename(columns={'time': 'ds', 'analysed_sst': 'y'})

# Initialize and fit model
model = Prophet(yearly_seasonality=True, daily_seasonality=False)
model.fit(prophet_df)

In [ ]:
future = model.make_future_dataframe(periods=5*365)  # 5 years ahead (daily)
forecast = model.predict(future)

In [ ]:
fig = model.plot(forecast)
plt.title('SST Forecast with Seasonality (Prophet Model)')
plt.xlabel('Date')
plt.ylabel('Analysed SST')
plt.grid(True)
plt.show()


In [ ]:
fig2 = model.plot_components(forecast)


| Plot   | Y-Axis Meaning                               | Effect Size   |
| ------ | -------------------------------------------- | ------------- |
| Weekly | SST deviation by day of week (±0.005 approx) | Tiny impact   |
| Yearly | SST deviation by time of year (±1.5 approx)  | Strong impact |


SST tends to be ~1.5 units above average during peak warm months, and ~1.2 units below average in the colder months — showing a strong seasonal oscillation.

In [ ]:
from prophet.diagnostics import cross_validation, performance_metrics
df_cv = cross_validation(model, initial='1095 days', period='180 days', horizon='365 days')
df_p = performance_metrics(df_cv)
df_p.head()